In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(),"apperception"))

### IMPORTS
import cv2

from world import *
from world_executor import *
from video_util import *
from metadata_util import *
import lens
import point



In [2]:
### Let's define some attribute for constructing the world first
name = 'traffic_scene' # world name
units = 'metrics'      # world units
video_file = "./amber_videos/traffic-scene-shorter.mp4" #example video file
lens_attrs = {'fov': 120, 
              'cam_origin': (0, 0, 0), 
              'skew_factor': 0}
point_attrs = {'p_id': 'p1', 
               'cam_id': 'cam1', 
               'x': 0,
               'y': 0, 
               'z': 0,
               'time': None, 
               'type':'pos'}
camera_attrs = {'ratio': 0.5}
fps = 30


In [3]:
### First we define a world
traffic_world = World(name=name, units=units)

### Use TASM if it's available on the machine
# traffic_world = World(name=name, units=units, enable_tasm=True)

In [4]:
### Secondly we construct the camera
fov, res, cam_origin, skew_factor = lens_attrs['fov'], [1280, 720], lens_attrs['cam_origin'], lens_attrs['skew_factor']
cam_lens = lens.PinholeLens(res, cam_origin, fov, skew_factor)

pt_id, cam_id, x, y, z, time, pt_type = point_attrs['p_id'], point_attrs['cam_id'], point_attrs['x'], point_attrs['y'], point_attrs['z'], point_attrs['time'], point_attrs['type']
location = point.Point(pt_id, cam_id, x, y, z, time, pt_type)

ratio = camera_attrs['ratio']

In [5]:
### Ingest the camera to the world
traffic_world = traffic_world.camera(cam_id=cam_id, 
                               location=location, 
                               ratio=ratio, 
                               video_file=video_file, 
                               metadata_identifier=name+"_"+cam_id, 
                               lens=cam_lens)

In [9]:
### Call execute on the world to run the detection algorithm and save the real data to the database
recognized_world = traffic_world.recognize(cam_id)
recognized_world.execute()

video 1/1 (1/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: 

/Users/byronhsu/Desktop/apperception/env/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


video 1/1 (2/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (3/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (4/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (5/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (6/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (7/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (8/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (9/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (10/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (11/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (12/123) /Users/byronhsu/Desktop/apper

video 1/1 (88/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (89/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (90/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (91/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (92/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (93/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (94/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (95/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (96/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (97/123) /Users/byronhsu/Desktop/apperception/amber_videos/traffic-scene-shorter.mp4: video 1/1 (98/123) /Users/byronhsu/Deskt

In [10]:
volume = traffic_world.select_intersection_of_interest_or_use_default(cam_id=cam_id)
filtered_world = traffic_world.predicate(lambda obj:obj.object_type == "car").predicate(lambda obj:obj.location in volume, {"volume":volume})
filtered_world = filtered_world.interval([0,fps*4])
 
### to get the trajectory and the video over the entire trajectory(amber case)
filtered_ids = filtered_world.selectkey(distinct = True).execute()
print("filtered_ids are", filtered_ids)
print(len(filtered_ids))
if len(filtered_ids)>0:
    id_array = [e[0] for e in filtered_ids]
    ### Fetch the trajectory of these items
    trajectory = traffic_world.predicate(lambda obj: obj.object_id in id_array, {"id_array":id_array}).get_trajectory(distinct=True).execute()
    traffic_world.overlay_trajectory(cam_id, trajectory)
    ### Get the videos of these items
    entire_video = traffic_world.predicate(lambda obj: obj.object_id in id_array, {"id_array":id_array}).get_video()
    entire_video.execute()

{'cam1': <video_context.Camera object at 0x7fe650562c40>}
Module(body=[Return(value=Compare(left=Attribute(value=Name(id='obj', ctx=Load()), attr='object_type', ctx=Load()), ops=[Eq()], comparators=[Constant(value='car', kind=None)]))], type_ignores=[])
Module(body=[Return(value=Compare(left=Attribute(value=Name(id='obj', ctx=Load()), attr='location', ctx=Load()), ops=[In()], comparators=[Name(id='volume', ctx=Load())]))], type_ignores=[])
Module(body=[Return(value=Compare(left=Call(func=Name(id='Tmin', ctx=Load()), args=[Attribute(value=Name(id='obj', ctx=Load()), attr='location', ctx=Load())], keywords=[]), ops=[GtE()], comparators=[Name(id='start', ctx=Load())]))], type_ignores=[])
Module(body=[Return(value=Compare(left=Call(func=Name(id='Tmax', ctx=Load()), args=[Attribute(value=Name(id='obj', ctx=Load()), attr='location', ctx=Load())], keywords=[]), ops=[Lt()], comparators=[Name(id='end', ctx=Load())]))], type_ignores=[])
SELECT distinct on(itemId) Item_General_Trajectory.itemId F

/Users/byronhsu/Desktop/apperception/env/lib/python3.8/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Module(body=[Return(value=Compare(left=Attribute(value=Name(id='obj', ctx=Load()), attr='object_id', ctx=Load()), ops=[In()], comparators=[Name(id='id_array', ctx=Load())]))], type_ignores=[])
Module(body=[Return(value=Compare(left=Call(func=Name(id='Tmin', ctx=Load()), args=[Attribute(value=Name(id='obj', ctx=Load()), attr='location', ctx=Load())], keywords=[]), ops=[GtE()], comparators=[Name(id='start', ctx=Load())]))], type_ignores=[])
Module(body=[Return(value=Compare(left=Call(func=Name(id='Tmax', ctx=Load()), args=[Attribute(value=Name(id='obj', ctx=Load()), attr='location', ctx=Load())], keywords=[]), ops=[Lt()], comparators=[Name(id='end', ctx=Load())]))], type_ignores=[])
Xmin(General_Bbox.trajBbox)
Ymin(General_Bbox.trajBbox)
Zmin(General_Bbox.trajBbox)
Xmax(General_Bbox.trajBbox)
Ymax(General_Bbox.trajBbox)
Zmax(General_Bbox.trajBbox)
Tmin(General_Bbox.trajBbox)
SELECT Item_General_Trajectory.itemId, Xmin(General_Bbox.trajBbox), Ymin(General_Bbox.trajBbox), Zmin(General_Bbox